In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline
import statsmodels.api as sm
import scipy.stats as st

In [3]:
url = '../../data/clean/combined_data_control.txt'
df = pd.read_csv(url)

In [6]:
# Ensure date_time is in datetime format
df['date_time'] = pd.to_datetime(df['date_time'])

In [8]:
# Sort by client_id, visit_id, and date_time to ensure chronological order
df = df.sort_values(by=['client_id',"visitor_id", "visit_id",'date_time'])

In [10]:
# Define the process steps in the correct order
process_order = ['start', 'step_1', 'step_2', 'step_3', 'confirm']

In [12]:
# Create a new column that shows the next step
df['next_step'] = df.groupby(['client_id',"visitor_id", "visit_id"])['process_step'].shift(-1)

In [14]:
# Identify step-back errors
df['is_step_back'] = df.apply(
    lambda row: process_order.index(row['next_step']) < process_order.index(row['process_step']) 
    if pd.notnull(row['next_step']) else False, axis=1
)

In [16]:
# Calculate total steps and total errors
total_steps = len(df)  # Total number of rows (steps)
total_step_back_errors = df['is_step_back'].sum()  # Sum of step-back errors
total_step_back_errors

9576

In [18]:
# Calculate the overall error rate
total_error_rate = total_step_back_errors / total_steps * 100

print(f"Total number of steps: {total_steps}")
print(f"Total number of step-back errors: {total_step_back_errors}")
print(f"Total error rate: {total_error_rate:.6f}%")

Total number of steps: 140482
Total number of step-back errors: 9576
Total error rate: 6.816532%


In [38]:
# 1) Filter and get the last 'start' and 'confirm' time for each client
start_times = (
    df[df['process_step'] == 'start']
    .groupby(['client_id', 'visitor_id', 'visit_id'])['date_time']
    .first()
    .reset_index(name='start_time')
)

confirm_times = (
    df[df['process_step'] == 'confirm']
    .groupby(['client_id', 'visitor_id', 'visit_id'])['date_time']
    .last()
    .reset_index(name='confirm_time')
)

In [40]:
start_times
# 30,955 rows 

,client_id,visitor_id,visit_id,start_time
0,1028,42237450_62128060588,557292053_87239438319_391157,2017-04-08 18:51:28
1,1104,194240915_18158000533,543158812_46395476577_767725,2017-06-12 07:49:18
2,1104,194240915_18158000533,643221571_99977972121_69283,2017-06-20 22:31:33
3,1186,446844663_31615102958,507052512_11309370126_442139,2017-04-08 15:59:16
4,1186,446844663_31615102958,795373564_99931517312_810896,2017-04-08 18:05:02
...,...,...,...,...
30950,9997470,395791369_55562604618,761490147_96352537762_21814,2017-05-09 15:55:48
30951,9997470,395791369_55562604618,904791598_9725982898_416914,2017-04-20 20:04:38
30952,9997470,91394485_75296404278,655572400_94971272893_411965,2017-04-07 16:11:03
30953,9998346,292425655_16607136645,189177304_69869411700_783154,2017-03-29 15:25:00


In [42]:
confirm_times
# 16,081 rows

,client_id,visitor_id,visit_id,confirm_time
0,1195,766842522_69992551638,393817425_39015278493_996341,2017-04-05 20:19:31
1,1197,753759429_54481946928,71862471_21202285428_848395,2017-04-01 17:17:59
2,3743,179439784_83920302761,438930656_9173628148_961548,2017-06-16 06:37:04
3,5354,233679354_36495397627,192604721_14077630095_819213,2017-04-12 20:31:31
4,6642,378857471_644475492,261034804_26267268728_251909,2017-04-19 12:27:58
...,...,...,...,...
16076,9996711,560523684_67875119133,200256953_19881035574_656660,2017-04-21 19:20:47
16077,9997114,708360389_95473182321,19940015_3197786491_51650,2017-04-14 20:21:31
16078,9997125,435478449_41974372268,983995660_11938398128_321832,2017-04-16 08:42:33
16079,9997470,395791369_55562604618,761490147_96352537762_21814,2017-05-09 16:39:34


In [45]:
# 2) Merge start and confirm times
process_times = pd.merge(start_times, confirm_times, on=['client_id', 'visitor_id', 'visit_id'], how='inner')
process_times = process_times.dropna()
process_times

,client_id,visitor_id,visit_id,start_time,confirm_time
0,1195,766842522_69992551638,393817425_39015278493_996341,2017-04-05 20:15:26,2017-04-05 20:19:31
1,1197,753759429_54481946928,71862471_21202285428_848395,2017-04-01 17:16:24,2017-04-01 17:17:59
2,3743,179439784_83920302761,438930656_9173628148_961548,2017-06-16 06:32:12,2017-06-16 06:37:04
3,5354,233679354_36495397627,192604721_14077630095_819213,2017-04-12 20:30:01,2017-04-12 20:31:31
4,6642,378857471_644475492,261034804_26267268728_251909,2017-04-19 12:20:35,2017-04-19 12:27:58
...,...,...,...,...,...
14931,9996711,560523684_67875119133,200256953_19881035574_656660,2017-04-21 19:09:21,2017-04-21 19:20:47
14932,9997114,708360389_95473182321,19940015_3197786491_51650,2017-04-14 20:16:33,2017-04-14 20:21:31
14933,9997125,435478449_41974372268,983995660_11938398128_321832,2017-04-16 08:39:35,2017-04-16 08:42:33
14934,9997470,395791369_55562604618,761490147_96352537762_21814,2017-05-09 15:55:48,2017-05-09 16:39:34


In [51]:
# 3) Calculate total time spent in seconds
process_times['total_time_spent'] = (process_times['confirm_time'] - process_times['start_time']).dt.total_seconds()
process_times.describe()

,client_id,start_time,confirm_time,total_time_spent
count,1.493600e+04,14936,14936,14936.000000
mean,5.025728e+06,2017-04-20 14:45:56.570098944,2017-04-20 14:52:36.595473920,400.025375
min,1.195000e+03,2017-03-15 00:19:28,2017-03-15 00:26:24,-1819.000000
25%,2.548779e+06,2017-04-04 10:10:54.249999872,2017-04-04 10:20:54.249999872,174.000000
50%,5.040721e+06,2017-04-12 12:37:13.500000,2017-04-12 12:41:00.500000,274.000000
75%,7.493655e+06,2017-05-02 14:42:28.750000128,2017-05-02 14:51:50.500000,460.000000
max,9.998346e+06,2017-06-20 23:13:35,2017-06-20 23:26:00,17072.000000
std,2.870821e+06,NaN,NaN,452.898750


In [56]:
# Average of time to complete the process (14936 rows)
process_times.total_time_spent.mean()

400.02537493304766

In [63]:
# Step 1: Calculate the average process_duration
avg_process_duration = process_times['total_time_spent'].mean()

# Step 2: Filter the DataFrame to include only rows where process_duration is greater than the average
filtered_df = process_times[process_times['total_time_spent'] > avg_process_duration]

# Step 3: Get the unique client_ids from the filtered DataFrame
clients_above_avg_duration = filtered_df['client_id'].unique()

total_clients_above_avg = len(clients_above_avg_duration)

# If you want to print or use the result
total_clients_above_avg

4569

In [65]:
min_duration = filtered_df['total_time_spent'].min()
min_duration
# 6.68 minutes 

401.0

In [67]:
max_duration = filtered_df['total_time_spent'].max()
max_duration
# 284.53 minutes 

17072.0

In [69]:
# Calculate the time difference between consecutive steps for each user
df['time_diff'] = df.groupby(['client_id', 'visitor_id', 'visit_id'])['date_time'].diff()
df['time_diff']
df

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,next_step,is_step_back,time_diff
8,1028,42237450_62128060588,557292053_87239438319_391157,start,2017-04-08 18:51:28,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_1,False,NaT
7,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:52:17,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_1,False,0 days 00:00:49
6,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:53:20,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_2,False,0 days 00:01:03
5,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 18:53:29,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_3,False,0 days 00:00:09
4,1028,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_1,True,0 days 00:04:35
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140473,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:30:11,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_3,False,0 days 00:00:16
140472,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:30:25,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_3,False,0 days 00:00:14
140470,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:37:28,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,confirm,False,0 days 00:07:03
140471,9998346,292425655_16607136645,189177304_69869411700_783154,confirm,2017-03-29 15:37:28,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,NaN,False,0 days 00:00:00


In [71]:
df['time_diff'].isna().sum()
# I have missing values, because the "start" is the first step. 

32235

In [73]:
df[df['client_id'] == 9998346	].head(10)

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,next_step,is_step_back,time_diff
140480,9998346,292425655_16607136645,189177304_69869411700_783154,start,2017-03-29 15:25:00,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_1,False,NaT
140479,9998346,292425655_16607136645,189177304_69869411700_783154,step_1,2017-03-29 15:26:02,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_2,False,0 days 00:01:02
140478,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:26:47,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_2,False,0 days 00:00:45
140477,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:27:40,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_2,False,0 days 00:00:53
140476,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:27:49,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_3,False,0 days 00:00:09
140475,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:28:00,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_1,True,0 days 00:00:11
140474,9998346,292425655_16607136645,189177304_69869411700_783154,step_1,2017-03-29 15:29:55,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_2,False,0 days 00:01:55
140473,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:30:11,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_3,False,0 days 00:00:16
140472,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:30:25,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_3,False,0 days 00:00:14
140470,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:37:28,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,confirm,False,0 days 00:07:03


In [77]:
df['time_diff_seconds'] = df['time_diff'].dt.total_seconds()
df

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,next_step,is_step_back,time_diff,time_diff_seconds
8,1028,42237450_62128060588,557292053_87239438319_391157,start,2017-04-08 18:51:28,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_1,False,NaT,NaN
7,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:52:17,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_1,False,0 days 00:00:49,49.0
6,1028,42237450_62128060588,557292053_87239438319_391157,step_1,2017-04-08 18:53:20,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_2,False,0 days 00:01:03,63.0
5,1028,42237450_62128060588,557292053_87239438319_391157,step_2,2017-04-08 18:53:29,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_3,False,0 days 00:00:09,9.0
4,1028,42237450_62128060588,557292053_87239438319_391157,step_3,2017-04-08 18:58:04,12.0,145.0,36.0,M,3.0,103520.22,1.0,4.0,Control,step_1,True,0 days 00:04:35,275.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140473,9998346,292425655_16607136645,189177304_69869411700_783154,step_2,2017-03-29 15:30:11,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_3,False,0 days 00:00:16,16.0
140472,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:30:25,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,step_3,False,0 days 00:00:14,14.0
140470,9998346,292425655_16607136645,189177304_69869411700_783154,step_3,2017-03-29 15:37:28,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,confirm,False,0 days 00:07:03,423.0
140471,9998346,292425655_16607136645,189177304_69869411700_783154,confirm,2017-03-29 15:37:28,50.0,602.0,61.5,F,2.0,149881.38,6.0,9.0,Control,NaN,False,0 days 00:00:00,0.0


In [79]:
# Group by 'client_id', 'visitor_id', and 'visit_id', sum the time for each group
total_time_per_visit = df.groupby(['client_id', 'visitor_id', 'visit_id'])['time_diff_seconds'].sum().reset_index()

# Now, sort by 'client_id', 'visit_id', and 'visitor_id'
total_time_per_visit_sorted = total_time_per_visit.sort_values(by=['client_id', 'visit_id', 'visitor_id'])

# If you want to see the result
print(total_time_per_visit_sorted)

       client_id             visitor_id                      visit_id  \
0           1028   42237450_62128060588  557292053_87239438319_391157   
1           1104  194240915_18158000533  543158812_46395476577_767725   
2           1104  194240915_18158000533   643221571_99977972121_69283   
3           1186  446844663_31615102958  507052512_11309370126_442139   
4           1186  446844663_31615102958  795373564_99931517312_810896   
...          ...                    ...                           ...   
32232    9997470   91394485_75296404278  655572400_94971272893_411965   
32230    9997470  395791369_55562604618   761490147_96352537762_21814   
32231    9997470  395791369_55562604618   904791598_9725982898_416914   
32233    9998346  292425655_16607136645  189177304_69869411700_783154   
32234    9998921  960034051_85687824805    378424178_59565976155_8051   

       time_diff_seconds  
0                  538.0  
1                    0.0  
2                    0.0  
3              

In [81]:
total_time_per_visit.head(20)

,client_id,visitor_id,visit_id,time_diff_seconds
0,1028,42237450_62128060588,557292053_87239438319_391157,538.0
1,1104,194240915_18158000533,543158812_46395476577_767725,0.0
2,1104,194240915_18158000533,643221571_99977972121_69283,0.0
3,1186,446844663_31615102958,507052512_11309370126_442139,0.0
4,1186,446844663_31615102958,795373564_99931517312_810896,22.0
5,1195,766842522_69992551638,393817425_39015278493_996341,245.0
6,1197,753759429_54481946928,71862471_21202285428_848395,95.0
7,1368,366307863_19014662045,784065271_45379483290_309335,0.0
8,2439,607208067_70160939111,848231744_22569944243_37711,0.0
9,2581,770616558_80928163524,182925466_27021409208_83502,0.0


In [83]:
total_time_per_visit_sorted

,client_id,visitor_id,visit_id,time_diff_seconds
0,1028,42237450_62128060588,557292053_87239438319_391157,538.0
1,1104,194240915_18158000533,543158812_46395476577_767725,0.0
2,1104,194240915_18158000533,643221571_99977972121_69283,0.0
3,1186,446844663_31615102958,507052512_11309370126_442139,0.0
4,1186,446844663_31615102958,795373564_99931517312_810896,22.0
...,...,...,...,...
32232,9997470,91394485_75296404278,655572400_94971272893_411965,0.0
32230,9997470,395791369_55562604618,761490147_96352537762_21814,2626.0
32231,9997470,395791369_55562604618,904791598_9725982898_416914,0.0
32233,9998346,292425655_16607136645,189177304_69869411700_783154,748.0


In [85]:
# Drop missing values 108,247 rows 
df = df.dropna(subset=['time_diff_seconds']).loc[df['time_diff_seconds'] != 0]
df.describe(include='all')

,client_id,visitor_id,visit_id,process_step,date_time,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,Variation,next_step,is_step_back,time_diff,time_diff_seconds
count,1.079160e+05,107916,107916,107916,107916,107916.000000,107916.000000,107916.000000,107916,107916.000000,1.079160e+05,107916.000000,107916.000000,107916,81544,107916,107916,107916.000000
unique,NaN,22833,26514,5,NaN,NaN,NaN,NaN,3,NaN,NaN,NaN,NaN,1,5,2,NaN,NaN
top,NaN,477423541_86323531833,866078022_75847251057_443309,step_1,NaN,NaN,NaN,NaN,U,NaN,NaN,NaN,NaN,Control,step_2,False,NaN,NaN
freq,NaN,47,27,29239,NaN,NaN,NaN,NaN,37638,NaN,NaN,NaN,NaN,107916,25339,98436,NaN,NaN
mean,5.010562e+06,NaN,NaN,NaN,2017-04-22 05:00:35.657845248,12.113783,151.358575,48.150367,NaN,2.270349,1.576259e+05,3.248740,6.284304,NaN,NaN,NaN,0 days 00:01:23.781598650,83.781599
min,1.028000e+03,NaN,NaN,NaN,2017-03-15 00:20:50,2.000000,33.000000,17.000000,NaN,2.000000,2.378961e+04,0.000000,3.000000,NaN,NaN,NaN,0 days 00:00:01,1.000000
25%,2.523295e+06,NaN,NaN,NaN,2017-04-05 11:14:32,6.000000,81.000000,34.500000,NaN,2.000000,4.091498e+04,1.000000,4.000000,NaN,NaN,NaN,0 days 00:00:13,13.000000
50%,5.026488e+06,NaN,NaN,NaN,2017-04-12 18:46:03,11.000000,136.000000,49.500000,NaN,2.000000,6.843123e+04,3.000000,6.000000,NaN,NaN,NaN,0 days 00:00:37,37.000000
75%,7.464330e+06,NaN,NaN,NaN,2017-05-09 12:24:12.500000,16.000000,193.000000,60.500000,NaN,2.000000,1.482921e+05,6.000000,9.000000,NaN,NaN,NaN,0 days 00:01:24,84.000000
max,9.998346e+06,NaN,NaN,NaN,2017-06-20 23:57:06,55.000000,669.000000,96.000000,NaN,7.000000,7.741994e+06,6.000000,9.000000,NaN,NaN,NaN,0 days 11:10:35,40235.000000


In [87]:
# Group by 'process_step' and calculate the average time spent in each step
avg_time_per_step = df.groupby('process_step')['time_diff_seconds'].mean().reset_index()
avg_time_per_step

,process_step,time_diff_seconds
0,confirm,130.692770
1,start,174.742134
2,step_1,43.199494
3,step_2,38.944379
4,step_3,94.862765


In [89]:
df_time_start= df[df["process_step"]=="start"]["time_diff_seconds"]
df_time_step1= df[df["process_step"]=="step_1"]["time_diff_seconds"]
df_time_step2= df[df["process_step"]=="step_2"]["time_diff_seconds"]
df_time_step3= df[df["process_step"]=="step_3"]["time_diff_seconds"]
df_time_confirm= df[df["process_step"]=="confirm"]["time_diff_seconds"]

In [91]:
#Set the hypothesis

#H0: mu_start time control = mu_confirm time control
#H1: mu_start time test != mu_confirm time group

#significance level = 0.05

st.ttest_ind(df_time_start,df_time_confirm, equal_var=False, alternative="two-sided")

TtestResult(statistic=9.381249768841386, pvalue=7.288306409409047e-21, df=17705.564670833934)

## Interpretation:
The t-statistic of 9.38 indicates a large difference between the groups (or means being compared).
The p-value of 7.28×10 −21 is extremely small, suggesting that the null hypothesis (no difference) can almost certainly be rejected. This means the result is highly significant statistically.
The degrees of freedom of approximately 17,679 suggests that the sample size is quite large.
## Conclusion:
Based on these results, the difference between the two groups being compared is statistically significant, with a very small probability that this result is due to random chance. You can confidently reject the null hypothesis and conclude that the groups (or conditions) differ in a meaningful way.

In [94]:
# Filter the rows where process_duration is less than 8.66 minutes 
filtered_visits = process_times[process_times['total_time_spent'] < 0]

# Sort the filtered DataFrame by 'process_duration' in ascending order
sorted_filtered_visits = filtered_visits.sort_values(by='total_time_spent')

# Now, if you want to get the visit_ids sorted by process_duration:
client_ids_sorted = sorted_filtered_visits['client_id']

# Print the sorted visit_ids
client_ids_sorted

6531     4416226
14691    9829438
13065    8723535
11925    7974545
7776     5253882
8647     5829248
2370     1643533
5729     3864935
12462    8319901
1312      908517
907       613664
31         21970
7902     5330356
10015    6741997
10285    6925389
276       180953
5843     3945005
Name: client_id, dtype: int64

In [96]:
sorted_filtered_visits.head(17)

,client_id,visitor_id,visit_id,start_time,confirm_time,total_time_spent
6531,4416226,155055665_78581777702,175695329_63350883005_788101,2017-04-24 15:52:04,2017-04-24 15:21:45,-1819.0
14691,9829438,75692063_34789201682,923209093_52642149345_96302,2017-04-10 21:57:21,2017-04-10 21:30:07,-1634.0
13065,8723535,689709595_60201005318,608878385_51120543756_227552,2017-04-14 12:08:03,2017-04-14 11:53:54,-849.0
11925,7974545,388632696_76940487835,878149558_45546903135_33404,2017-04-20 13:35:22,2017-04-20 13:26:03,-559.0
7776,5253882,209491381_20082544640,648083630_90353902947_519109,2017-04-07 12:49:33,2017-04-07 12:41:04,-509.0
8647,5829248,15888053_13481691729,920201364_61747349410_114522,2017-05-23 13:53:39,2017-05-23 13:48:30,-309.0
2370,1643533,353982891_80880835112,171886830_38696361268_917175,2017-04-04 10:43:49,2017-04-04 10:38:51,-298.0
5729,3864935,621704638_62794476112,890284055_564025159_169962,2017-04-05 14:53:19,2017-04-05 14:49:58,-201.0
12462,8319901,152559771_57408948187,295319191_49098444170_571382,2017-04-13 06:54:16,2017-04-13 06:51:01,-195.0
1312,908517,37268744_19019020116,608220588_80141434977_777946,2017-04-20 14:56:08,2017-04-20 14:54:40,-88.0


In [98]:
# Filter the rows where visit_id matches the specific visit_id
client_id_to_check = 3945005	
filter_df = df[df['client_id'] == client_id_to_check]

# To display the filtered data
print(filter_df)

       client_id             visitor_id                    visit_id  \
54919    3945005  340487723_50109925261  638857358_8100128732_84471   
54918    3945005  340487723_50109925261  638857358_8100128732_84471   
54917    3945005  340487723_50109925261  638857358_8100128732_84471   
54916    3945005  340487723_50109925261  638857358_8100128732_84471   
54915    3945005  340487723_50109925261  638857358_8100128732_84471   

      process_step           date_time  clnt_tenure_yr  clnt_tenure_mnth  \
54919       step_2 2017-04-06 12:57:16            15.0             184.0   
54918       step_3 2017-04-06 12:58:35            15.0             184.0   
54917      confirm 2017-04-06 12:59:02            15.0             184.0   
54916        start 2017-04-06 12:59:26            15.0             184.0   
54915       step_1 2017-04-06 12:59:30            15.0             184.0   

       clnt_age gendr  num_accts        bal  calls_6_mnth  logons_6_mnth  \
54919      49.5     M        2.0  160072

In [100]:
# Filter the rows where process_duration is less than 400 seconds  
filtered_visits2 = process_times[process_times['total_time_spent'] > 400.942355]

# Sort the filtered DataFrame by 'process_duration' in ascending order
sorted_filtered_visits2 = filtered_visits2.sort_values(by='total_time_spent')

# Now, if you want to get the visit_ids sorted by process_duration:
client_ids_sorted2 = sorted_filtered_visits2['client_id']

# Print the sorted visit_ids
client_ids_sorted2

3140     2161349
4888     3338846
4978     3400860
10039    6754992
10035    6753738
          ...   
6477     4379554
3342     2283351
4212     2882702
9507     6420672
7818     5282553
Name: client_id, Length: 4571, dtype: int64

# 4,571 clients with the total time spent greater than the average (401)
# 11 clients with the total time equal to the average (400)
# 10,365 clients with the total time spent less than the average (400.94)

In [103]:
# Step 1: Filter out rows where 'total_time_spent' is negative
# 14,919 rows 
df_clean = process_times[process_times['total_time_spent'] >= 0]
df_clean.describe()

,client_id,start_time,confirm_time,total_time_spent
count,1.491900e+04,14919,14919,14919.000000
mean,5.026057e+06,2017-04-20 14:54:13.661036032,2017-04-20 15:00:54.603391744,400.942355
min,1.195000e+03,2017-03-15 00:19:28,2017-03-15 00:26:24,33.000000
25%,2.549167e+06,2017-04-04 09:59:43,2017-04-04 10:05:30,174.000000
50%,5.040709e+06,2017-04-12 12:38:02,2017-04-12 12:43:01,275.000000
75%,7.493718e+06,2017-05-02 15:13:02.500000,2017-05-02 15:16:02.500000,460.500000
max,9.998346e+06,2017-06-20 23:13:35,2017-06-20 23:26:00,17072.000000
std,2.870555e+06,NaN,NaN,451.986091


In [105]:
minutes= 400.942355/60
minutes

6.682372583333334